<a href="https://colab.research.google.com/github/lynnkuan/ML/blob/main/randomforest%26xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
titanic = pd.read_csv('titanic.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
titanic['Age'].fillna(titanic['Age'].median(skipna=True),inplace=True)
titanic['Embarked'].fillna('S',inplace=True)

titanic['TravelAlone'] = np.where((titanic['SibSp']+titanic['Parch'])>0,0,1)

titanic.drop('Cabin',axis=1,inplace=True)
titanic.drop('SibSp',axis=1,inplace=True)
titanic.drop('Parch',axis=1,inplace=True)
titanic.drop('PassengerId',axis=1,inplace=True)
titanic.drop('Name',axis=1,inplace=True)
titanic.drop('Ticket',axis=1,inplace=True)

labelencoder = LabelEncoder()
titanic['Sex'] = labelencoder.fit_transform(titanic['Sex'])
titanic['Embarked'] = labelencoder.fit_transform(titanic['Embarked'])
titanic

<ipython-input-5-c9d11dfbd722>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['Age'].fillna(titanic['Age'].median(skipna=True),inplace=True)
<ipython-input-5-c9d11dfbd722>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

,Survived,Pclass,Sex,Age,Fare,Embarked,TravelAlone
0,0,3,1,22.0,7.2500,2,0
1,1,1,0,38.0,71.2833,0,0
2,1,3,0,26.0,7.9250,2,1
3,1,1,0,35.0,53.1000,2,0
4,0,3,1,35.0,8.0500,2,1
...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,2,1
887,1,1,0,19.0,30.0000,2,1
888,0,3,0,28.0,23.4500,2,0
889,1,1,1,26.0,30.0000,0,1


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(titanic.iloc[:,1:],titanic['Survived'],
                                                 test_size=0.3,random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,auc,precision_score,recall_score,f1_score,confusion_matrix,roc_curve

model = RandomForestClassifier()
model.fit(x_train,y_train)
pred = model.predict(x_test)

cm = confusion_matrix(y_test,pred)
fpr,tpr,thresholds = roc_curve(y_test,pred)

metrics = {'Accuracy':accuracy_score(y_test,pred),'AUC':auc(fpr,tpr),'precision score':precision_score(y_test,pred),'recall score':recall_score(y_test,pred),'f1 score':f1_score(y_test,pred)}
df = pd.DataFrame([metrics])
df

,Accuracy,AUC,precision score,recall score,f1 score
0,0.794776,0.785247,0.764151,0.72973,0.746544


XGBoost

In [13]:
#xgboost regressor
import xgboost as xgb

'''
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(x_train,y_train)
pred = model.predict(x_test)

r2_score,mean_absolute_error...
'''

"\nmodel = xgb.XGBRegressor(objective='reg:squarederror')\nmodel.fit(x_train,y_train)\npred = model.predict(x_test)\n\nr2_score,mean_absolute_error...\n"

In [14]:
_train,x_test,y_train,y_test = train_test_split(titanic.iloc[:,1:],titanic['Survived'],
                                                 test_size=0.3,random_state=42)


In [15]:
model = xgb.XGBClassifier(objective='binary:logistic')
model.fit(x_train,y_train)
pred = model.predict(x_test)

fpr,tpr,thresholds = roc_curve(y_test,pred)

metrics = {'Accuracy':accuracy_score(y_test,pred),'AUC':auc(fpr,tpr),'precision score':precision_score(y_test,pred),'recall score':recall_score(y_test,pred),'f1 score':f1_score(y_test,pred)}
df = pd.DataFrame([metrics])
df

,Accuracy,AUC,precision score,recall score,f1 score
0,0.783582,0.771734,0.757282,0.702703,0.728972
